In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG
from tensorflow.keras.layers import Dense,  GlobalAveragePooling2D,Dense
import matplotlib.pyplot as plt
import itertools
from tensorflow.keras.applications.mobilenet import MobileNet

# print("TensorFlow Version:", tf.__version__)

In [ ]:
WORK_DIR = 'C:/Users/LENOVO/OneDrive/Desktop/SOURCE CODE 2/model/Dataset'

CLASSES = [ 'Healthy',
            'Others'
            'Yellow rust' ]

IMG=128

IMAGE_SIZE = [128, 128]
DIM = (IMG,IMG )

In [ ]:
ZOOM = [.99, 1.01]
BRIGHT_RANGE = [0.8, 1.2]
HORZ_FLIP = True
FILL_MODE = "constant"
DATA_FORMAT = "channels_last"

work_dr = IDG(rescale = 1./255, brightness_range=BRIGHT_RANGE, zoom_range=ZOOM, data_format=DATA_FORMAT, fill_mode=FILL_MODE, 
              horizontal_flip=HORZ_FLIP)

train_data_gen = work_dr.flow_from_directory(directory=WORK_DIR, target_size=DIM, batch_size=8939, shuffle=False)

In [ ]:
train_data, train_labels = train_data_gen.next()

In [ ]:
print(train_data.shape, train_labels.shape)

In [ ]:

train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size = 0.3, random_state=42)

In [ ]:
print(train_data.shape, train_labels.shape)

In [ ]:
print(val_data.shape, val_labels.shape)

In [ ]:
base_model = MobileNet(weights = 'imagenet', include_top=False, input_shape=(128, 128, 3))

In [ ]:
for layer in base_model.layers[-30:]:
    layer.trainable = True

In [ ]:
base_model.summary()

In [ ]:
mobilenet = tf.keras.models.Sequential()
mobilenet.add(base_model)
mobilenet.add(GlobalAveragePooling2D())
mobilenet.add(Dense(3, activation = 'softmax'))
mobilenet.summary()

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
mobilenet.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = mobilenet.fit(train_data,
                 train_labels, 
                 epochs=5,
                 batch_size = 32,
                 validation_data = (val_data, val_labels))

In [ ]:
mobilenet.save('wheat_fine_tuned.h5')

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:

# Predict the values from the validation dataset
Y_pred = mobilenet.predict(val_data)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(val_labels,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(3))

In [ ]:
from sklearn.metrics import accuracy_score
acc=accuracy_score(Y_true,Y_pred_classes) 
print('Accuracy: %.3f' % acc)
from sklearn.metrics import precision_score
precision = precision_score(Y_true,Y_pred_classes,labels=[1,2], average='micro')
print('Precision: %.3f' % precision)
from sklearn.metrics import recall_score
recall = recall_score(Y_true,Y_pred_classes, average='micro')
print('Recall: %.3f' % recall)
from sklearn.metrics import f1_score
score = f1_score(Y_true,Y_pred_classes, average='micro')
print('F-Measure: %.3f' % score)

In [ ]:
history_dict = history.history

loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)

line1 = plt.plot(epochs, val_loss_values, label='Validation/Test Loss')
line2 = plt.plot(epochs, loss_values, label='Training Loss')
plt.setp(line1, linewidth=2.0, marker = '+', markersize=10.0)
plt.setp(line2, linewidth=2.0, marker = '4', markersize=10.0)
plt.xlabel('Epochs') 
plt.ylabel('Loss')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
history_dict = history.history

acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, len(loss_values) + 1)

line1 = plt.plot(epochs, val_acc_values, label='Validation/Test Accuracy')
line2 = plt.plot(epochs, acc_values, label='Training Accuracy')
plt.setp(line1, linewidth=2.0, marker = '+', markersize=10.0)
plt.setp(line2, linewidth=2.0, marker = '4', markersize=10.0)
plt.xlabel('Epochs') 
plt.ylabel('Accuracy')
plt.grid(True)
plt.legend()
plt.show()